In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer

# Graphs
import matplotlib.pyplot as plt

# Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from imblearn.over_sampling import SMOTE

# Metrics
from sklearn.metrics import roc_curve, roc_auc_score, auc, ConfusionMatrixDisplay, classification_report, confusion_matrix
from sklearn import tree

# SHAP values
import shap

In [ ]:
def data_prep(df, topogrup):
    """
    Prepares the dataset for analysis by applying filters and transformations.

    Args:
        df (pandas.DataFrame): The input DataFrame containing the raw data.
        topogrup (list): A list of topography group codes to filter the data.

    Returns:
        pandas.DataFrame: The prepared DataFrame ready for analysis.

    """

    df_aux = df.copy()

    # Filter data based on topography group
    df_aux = df_aux[df_aux.TOPOGRUP.isin(topogrup)]

    # Filter for individuals aged 19 and above
    df_aux = df_aux[df_aux.IDADE > 19]

    # Filter for residents of the state of SP
    df_aux = df_aux[df_aux.UFRESID == 'SP']

    # Filter out specific ECGRUP values
    df_aux = df_aux[~df_aux.ECGRUP.isin(['0','X','Y'])]

    # Filter for cases with microscopic confirmation
    df_aux = df_aux[df_aux.BASEDIAG == 3]

    # Convert date columns to datetime objects
    list_datas = ['DTCONSULT', 'DTDIAG', 'DTTRAT', 'DTULTINFO']
    for col_data in list_datas:
        df_aux[col_data] = pd.to_datetime(df_aux[col_data])

    # Calculate time differences in days
    df_aux['CONSDIAG'] = (df_aux.DTDIAG - df_aux.DTCONSULT).dt.days
    df_aux['DIAGTRAT'] = (df_aux.DTTRAT - df_aux.DTDIAG).dt.days
    df_aux['TRATCONS'] = (df_aux.DTTRAT - df_aux.DTCONSULT).dt.days
    df_aux['ULTIDIAG'] = (df_aux.DTULTINFO - df_aux.DTDIAG).dt.days
    df_aux[['DIAGTRAT', 'TRATCONS']] = df_aux[['DIAGTRAT', 'TRATCONS']].fillna(-1)

    # Categorize time differences
    df_aux.CONSDIAG = [0 if consdiag <= 30 else 1 if consdiag <= 60 else 2 for consdiag in df_aux.CONSDIAG]
    df_aux.TRATCONS = [99 if tratcons < 0 else 0 if tratcons <= 60 else 1 if tratcons <= 90 else 2 for tratcons in df_aux.TRATCONS]
    df_aux.DIAGTRAT = [99 if diagtrat < 0 else 0 if diagtrat <= 60 else 1 if diagtrat <= 90 else 2 for diagtrat in df_aux.DIAGTRAT]

    # Impute missing values in ESCOLARI using KNN
    cols_input = ['ESCOLARI', 'IDADE', 'ECGRUP', 'SEXO']

    df_esc = df_aux[cols_input].copy()
    df_esc.loc[df_esc.ESCOLARI == 9, 'ESCOLARI'] = np.nan
    df_esc.loc[df_esc.ECGRUP == 'I', 'ECGRUP'] = 1
    df_esc.loc[df_esc.ECGRUP == 'II', 'ECGRUP'] = 2
    df_esc.loc[df_esc.ECGRUP == 'III', 'ECGRUP'] = 3
    df_esc.loc[df_esc.ECGRUP == 'IV', 'ECGRUP'] = 4

    X = df_esc.values
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    df_aux['ESCOLARI_preench'] = imputer.fit_transform(X)[:,0].round()

    # Create a new feature indicating if treatment was in the same city
    df_aux['IBGE_idem_IBGEATEN'] = 0
    df_aux.loc[df_aux.IBGE == df_aux.IBGEATEN, 'IBGE_idem_IBGEATEN'] = 1

    # Create a new column 'presenca_rec' to indicate the presence of recurrence
    # 0: No recurrence
    # 1: Recurrence
    df_aux['presenca_rec'] = [0 if rec == 1 else 1 for rec in df_aux.RECNENHUM]

    # Correct for cases where RECNENHUM is 0 but DTRECIDIVA is not null
    df_aux.loc[(df_aux.presenca_rec == 0) & (df_aux.DTRECIDIVA.notnull()), 'presenca_rec'] = 1

    # Extract only the number from the DRS column
    DRS_expand = df_aux.DRS.str.split(' ', expand=True)
    df_aux['DRS'] = DRS_expand[1]

    # Drop unnecessary columns
    cols = df_aux.columns
    drop_cols = ['ESCOLARI', 'UFNASC', 'UFRESID', 'CIDADE', 'DTCONSULT', 'CLINICA',
                 'DTDIAG', 'BASEDIAG', 'DESCTOPO', 'MORFO', 'DESCMORFO', 'ECGRUP',
                 'T', 'N', 'M', 'PT', 'PN', 'PM', 'S', 'G', 'LOCALTNM', 'IDMITOTIC',
                 'PSA', 'GLEASON', 'OUTRACLA', 'META01', 'META02', 'META03',
                 'META04', 'DTTRAT', 'NAOTRAT', 'TRATAMENTO', 'TRATFANTES',
                 'TRATFAPOS', 'NENHUMANT', 'CIRURANT', 'RADIOANT', 'QUIMIOANT',
                 'HORMOANT', 'TMOANT', 'IMUNOANT', 'OUTROANT', 'NENHUMAPOS',
                 'CIRURAPOS', 'RADIOAPOS', 'QUIMIOAPOS', 'HORMOAPOS', 'TMOAPOS',
                 'IMUNOAPOS', 'OUTROAPOS', 'DTULTINFO', 'CICI', 'CICIGRUP',
                 'CICISUBGRU', 'FAIXAETAR', 'LATERALI', 'INSTORIG', 'PERDASEG',
                 'ERRO', 'DTRECIDIVA', 'RECNENHUM', 'RECLOCAL', 'RECREGIO',
                 'RECDIST', 'REC01', 'REC02', 'REC03', 'REC04', 'CIDO', 'DSCCIDO',
                 'HABILIT', 'HABIT11', 'HABILIT1', 'CIDADEH']  # List of columns to be dropped
    cols = cols.drop(drop_cols)

    return df_aux[cols]

#######################################################################################

def get_labels(df):
    """
    Creates new columns indicating the patient's survival status.

    Args:
        df (pandas.DataFrame): A DataFrame containing patient data, including
            'ULTINFO' (last status indicator) and 'ULTIDIAG' (time in days since diagnosis).

    Returns:
        pandas.DataFrame: A DataFrame with the following added columns:
            - 'obito_geral': Indicates whether the patient died (1) or not (0).
            - 'sobrevida_ano1': Indicates if the patient survived at least 1 year after diagnosis.
            - 'sobrevida_ano3': Indicates if the patient survived at least 3 years after diagnosis.
            - 'sobrevida_ano5': Indicates if the patient survived at least 5 years after diagnosis.
    """

    df_aux = df.copy()

    # Initialize survival status columns with 0 (event not occurred)
    df_aux['obito_geral'] = 0
    df_aux['sobrevida_ano1'] = 0
    df_aux['sobrevida_ano3'] = 0
    df_aux['sobrevida_ano5'] = 0

    # Define overall death based on ULTINFO
    df_aux.loc[df_aux.ULTINFO > 2, 'obito_geral'] = 1

    # Define survival at 1, 3, and 5 years based on ULTIDIAG (assuming ULTIDIAG is in days)
    df_aux.loc[df_aux.ULTIDIAG > 365, 'sobrevida_ano1'] = 1
    df_aux.loc[df_aux.ULTIDIAG > 3 * 365, 'sobrevida_ano3'] = 1
    df_aux.loc[df_aux.ULTIDIAG > 5 * 365, 'sobrevida_ano5'] = 1

    # Drop original columns 'ULTINFO' and 'ULTIDIAG'
    drop_cols = ['ULTINFO', 'ULTIDIAG']

    return df_aux.drop(columns=drop_cols)

#######################################################################################

def get_train_test(df, drop_cols, label, test_size=0.25, random_state=0):
    """
    Splits a DataFrame into training and testing sets.

    Args:
        df (pandas.DataFrame): The DataFrame to be split.
        drop_cols (list): A list of column names to be dropped.
        label (str): The name of the target variable.
        test_size (float, optional): The proportion of the dataset to include in the test split. Defaults to 0.25.
        random_state (int, optional): Controls the shuffling applied to the data before applying the split.
                                      Pass an int for reproducible output across multiple function calls. Defaults to 0.

    Returns:
        tuple: A tuple containing four DataFrames:
            X_train: Training set features.
            X_test: Testing set features.
            y_train: Training set labels.
            y_test: Testing set labels.
    """

    df_aux = df.copy()

    # Select features and target variable
    features = df_aux.drop(columns=drop_cols)
    target = df_aux[label]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    stratify=target)

    return X_train, X_test, y_train, y_test

#######################################################################################

def train_preprocessing(df, ohe_encoder=None, normalizer='StandardScaler'):
    """
    Performs preprocessing on the training dataset.

    Args:
        df (pandas.DataFrame): The DataFrame to be preprocessed.
        ohe_encoder (list, optional): A list of columns to apply one-hot encoding. If None,
            all categorical columns will be encoded using LabelEncoder.
        normalizer (str, optional): The type of normalization to apply. Defaults to 'StandardScaler'.
            Options include: 'StandardScaler', 'MinMaxScaler', 'MaxAbsScaler', 'QuantileTransformer'.

    Returns:
        tuple: A tuple containing:
            - df_aux (pandas.DataFrame): The preprocessed DataFrame.
            - enc (dict): A dictionary of encoders, one for each categorical variable.
            - norm: The normalization object used.
            - feat_cols (list): A list of feature column names.
    """

    df_aux = df.copy()

    enc = {}  # Dictionary to store encoders
    if ohe_encoder:
        # Apply one-hot encoding to specified columns
        for col in ohe_encoder:
            enc[col] = OneHotEncoder(handle_unknown='ignore', drop='first')
            ohe_results = enc[col].fit_transform(df_aux[[col]])
            # ... (rest of the one-hot encoding logic)

    # Apply label encoding to remaining categorical columns
    list_categorical = df_aux.select_dtypes(include='object').columns
    for col in list_categorical:
        enc[col] = LabelEncoder()
        df_aux[col] = enc[col].fit_transform(df_aux[col])

    # Select feature columns
    feat_cols = df_aux.columns

    # Apply normalization
    if normalizer == 'StandardScaler':
        norm = StandardScaler()
    elif normalizer == 'MinMaxScaler':
        norm = MinMaxScaler((0, 1))
    elif normalizer == 'MaxAbsScaler':
        norm = MaxAbsScaler()
    elif normalizer == 'QuantileTransformer':
        norm = QuantileTransformer(output_distribution='normal')

    df_aux = norm.fit_transform(df_aux)

    return df_aux, enc, norm, feat_cols

#######################################################################################

def test_preprocessing(df, enc, norm, ohe_encoder=None):
    """
    Performs preprocessing on the test dataset using the provided encoders and normalizer.

    Args:
        df (pandas.DataFrame): The DataFrame to be preprocessed.
        enc (dict): A dictionary of encoders used for categorical features in the training set.
        norm: The normalization object used for the training set.
        ohe_encoder (list, optional): A list of columns to apply one-hot encoding. If None,
            all categorical columns will be encoded using LabelEncoder.

    Returns:
        pandas.DataFrame: The preprocessed DataFrame.
    """

    df_aux = df.copy()

    if ohe_encoder:
        # Apply one-hot encoding to specified columns using pre-trained encoders
        for col in ohe_encoder:
            ohe_results = enc[col].transform(df_aux[[col]])
            df1 = pd.DataFrame(ohe_results.toarray(),
                               columns=enc[col].get_feature_names_out(),
                               index=df_aux[col].index)
            df_aux = df_aux.merge(df1, how='left', left_index=True, right_index=True)

        df_aux.drop(columns=ohe_encoder, inplace=True)

    # Apply label encoding to remaining categorical columns
    # Handle unseen categories by assigning them a new category (e.g., -1)
    list_categorical = df_aux.select_dtypes(include='object').columns
    for col in list_categorical:
        df_aux.loc[~df_aux[col].isin(enc[col].classes_), col] = -1
        df_aux.loc[df_aux[col].isin(enc[col].classes_), col] = enc[col].transform(df_aux[col][df_aux[col].isin(enc[col].classes_)])

    # Apply normalization using the pre-trained scaler
    df_aux = norm.transform(df_aux)

    return df_aux

#######################################################################################

def preprocessing(df, cols_drop, label, test_size=0.25, ohe_encoder=None,
                  norm_name='StandardScaler', return_enc_norm=False,
                  balance_data=False, random_state=0):
    """
    Performs comprehensive data preprocessing, including train-test split,
    categorical encoding, normalization, and optional class balancing.

    Args:
        df: The DataFrame containing the raw data.
        cols_drop: A list of columns to be dropped.
        label: The name of the target variable.
        test_size: The proportion of the dataset to include in the test split.
        ohe_encoder: A list of columns for one-hot encoding.
        norm_name: The name of the normalizer to use.
        return_enc_norm: Whether to return the encoders and normalizer.
        balance_data: Whether to balance the dataset using SMOTE.
        random_state: Random seed for reproducibility.

    Returns:
        A tuple containing:
            - X_train_: Preprocessed training features.
            - X_test_: Preprocessed testing features.
            - y_train_: Training labels.
            - y_test_: Testing labels.
            - feat_cols: Feature column names after preprocessing.
            - enc (optional): Dictionary of encoders used.
            - norm (optional): Normalization object used.
    """

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = get_train_test(df_aux, cols_drop, label,
                                                      test_size,
                                                      random_state=random_state)

    # Preprocess training set
    X_train_enc, enc, norm, feat_cols = train_preprocessing(X_train, ohe_encoder=ohe_encoder,
                                                            normalizer=norm_name)
    # Preprocess testing set using the same encoders and normalizer
    X_test_ = test_preprocessing(X_test, enc, norm, ohe_encoder=ohe_encoder)

    # Balance training data if specified
    if balance_data:
        X_train_, y_train_ = SMOTE(random_state=random_state).fit_resample(X_train_enc, y_train)

    # Print shapes for verification
    print(f'X_train = {X_train_.shape}, X_test = {X_test_.shape}')
    print(f'y_train = {y_train_.shape}, y_test = {y_test.shape}')

    # Return results
    if return_enc_norm:
        return X_train_, X_test_, y_train_, y_test, feat_cols, enc, norm
    else:
        return X_train_, X_test_, y_train_, y_test, feat_cols

#######################################################################################

def show_tree(model, feat_cols, max_depth=3, estimator=0):
    """
    Plots a decision tree from a RandomForestClassifier.

    Args:
        model (RandomForestClassifier): The trained RandomForestClassifier model.
        feat_cols (array-like): A list or array of feature names used in training.
        max_depth (int, optional): The maximum depth of the tree to display. Defaults to 3.
        estimator (int, optional): The index of the estimator to display. Defaults to 0.

    Returns:
        None: Displays the plotted tree.
    """

    # Create a figure for the plot
    plt.figure(figsize=(22, 10))

    # Plot the specified tree from the Random Forest
    tree.plot_tree(model.estimators_[estimator],
                  feature_names=feat_cols,
                  filled=True,  # Color leaf nodes
                  max_depth=max_depth)
    plt.show()  # Display the plot

#######################################################################################

def plot_feat_importances(model, feat_cols, n=10):
    """
    Plots a horizontal bar chart of the top n most important features in a model.

    Args:
        model: The trained machine learning model.
        feat_cols: A list or array of feature names.
        n: The number of top features to plot. Defaults to 10.

    Returns:
        None: Displays the plot.
    """

    # Get feature importances from the model
    feat_import = pd.Series(model.feature_importances_, index=feat_cols)

    # Plot the top n features
    feat_import.nlargest(n).plot(kind='barh', figsize=(10, 8))
    plt.show()

#######################################################################################

def plot_roc_curve(model, X_train, X_test, y_train, y_test):
    """
    Plots the Receiver Operating Characteristic (ROC) curve for a classification model,
    calculating the Area Under the Curve (AUC) for both training and testing sets.

    Args:
        model: A trained classification model with a `predict_proba` method.
        X_train: Training features.
        X_test: Testing features.
        y_train: Training labels.
        y_test: Testing labels.

    Returns:
        None: Displays the ROC curve plot.
    """

    # Get predicted probabilities for positive class
    probas_train = model.predict_proba(X_train)[:, 1]
    probas_test = model.predict_proba(X_test)[:, 1]

    # Calculate true positive rate (TPR) and false positive rate (FPR)
    fp_train, tp_train, _ = roc_curve(y_train, probas_train)
    fp_test, tp_test, _ = roc_curve(y_test, probas_test)

    # Calculate AUC
    auc_train = auc(fp_train, tp_train)
    auc_test = auc(fp_test, tp_test)

    # Plot ROC curves
    plt.figure(figsize=(10, 7))
    plt.plot(fp_train, tp_train, 'b', label=f'Train (AUC = {auc_train:.4f})')
    plt.plot(fp_test, tp_test, 'r', label=f'Test (AUC = {auc_test:.4f})')

    # Plot random guessing line
    plt.plot(np.linspace(0, 1, 100),
             np.linspace(0, 1, 100),
             label='Baseline',
             linestyle='--',
             color='k')

    # Set plot labels and display
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.grid(True)
    plt.legend()
    plt.show()

#######################################################################################

def plot_confusion_matrix(model, x, y, format='.5f', norm=True):
    """
    Plots a normalized confusion matrix and classification report for a given model.

    Args:
        model: A trained classification model.
        x: The features of the test set.
        y: The true labels of the test set.
        format (str, optional): Format specifier for the values in the confusion matrix.
            Defaults to '.5f' (5 decimal places).
        norm (bool, optional): If True, normalizes the confusion matrix. Defaults to True.

    Returns:
        None: Displays the confusion matrix and classification report.
    """

    # Create a confusion matrix plot
    with plt.rc_context({'font.size': 12, 'font.weight': 'bold'}):
        if norm:
            # Plot a normalized confusion matrix
            ConfusionMatrixDisplay.from_estimator(model, x, y, values_format=format,
                                                 cmap='binary', normalize='true')
        else:
            # Plot a non-normalized confusion matrix
            ConfusionMatrixDisplay.from_estimator(model, x, y, cmap='binary',
                                                 values_format='.0f')
        plt.show()

    # Generate a classification report
    pred = model.predict(x)
    print(f'\n{classification_report(y, pred, digits=5)}')

#######################################################################################

def plot_shap_values(model, x, features, max_display=10):
    """
    Plots a summary plot of SHAP values to visualize feature importance.

    Args:
        model: A trained machine learning model.
        x: A pandas DataFrame containing the features used to train the model.
        features: A list of feature names.
        max_display: The maximum number of features to display in the plot.

    Returns:
        None: Displays the SHAP summary plot.
    """

    # Calculate SHAP values
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(x)

    # Plot summary plot
    try:
        # For models with multiple outputs
        shap.summary_plot(shap_values[1], x, feature_names=features, max_display=max_display)
    except AssertionError:
        # For models with a single output
        shap.summary_plot(shap_values, x, feature_names=features, max_display=max_display)

#######################################################################################

def pred_cruzada(df, model, list_drop, label, enc, norm):
    """
    Performs cross-validation and plots the confusion matrix.

    Args:
        df: The DataFrame containing the data.
        model: A trained classification model.
        list_drop: A list of columns to drop.
        label: The name of the target variable.
        enc: A dictionary of encoders used for preprocessing.
        norm: The normalization object used.

    Returns:
        None: Displays the confusion matrix.
    """

    # Filter data for survival analysis
    df_sobrevida = df[~((df.obito_geral == 0) & (df[label] == 0))]
    df_sobrevida.reset_index(drop=True, inplace=True)

    # Extract target variable
    y = df_sobrevida[label].values

    # Extract features and preprocess
    X = df_sobrevida.drop(columns=list_drop)
    X_pp = test_preprocessing(X, enc, norm)

    # Plot confusion matrix
    plot_confusion_matrix(model, X_pp, y)